In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from pathlib import Path
import os
import re
import numpy as np

In [3]:
cur_path = Path(os.getcwd())
raw = cur_path / '..' / '..' / 'data' / '01_raw'

In [4]:
df = pd.read_csv(raw / 'sneaker.csv')
shoes = list(df.style_code)

In [30]:
def get_random_ua():
    random_ua = ''
    ua_file = raw / 'web-drivers' / 'user-agents.txt'
    try:
        with open(ua_file) as f:
            lines = f.readlines()
        if len(lines) > 0:
            prng = np.random.RandomState()
            index = prng.permutation(len(lines) - 1)
            idx = np.asarray(index, dtype=np.integer)[0]
            random_ua = lines[int(idx)]
            random_ua = random_ua[:len(random_ua)-2]
    except Exception as ex:
        print('Exception in random_ua')
        print(str(ex))
    finally:
        return random_ua

In [5]:
user_agent = get_random_ua()
options = Options()
options.add_argument(f'user-agent={user_agent}')
driver = webdriver.Chrome(options=options, executable_path= raw /'web-drivers'/ 'chromedriver')
driver.get('https://www.goat.com/')

In [7]:
url = 'https://www.goat.com/search?query=' + str(shoes[0])

In [35]:
user_agent = get_random_ua()
headers = {'user-agent': user_agent}
source = BeautifulSoup(requests.get(url, headers=headers).text)

In [ ]:
<span class="ProductTemplatePriceDisplay__LowestPrice-nfz87-3 dHaKT" data-qa="grid-cell-price">$461</span>

In [36]:
source

<!DOCTYPE html>

<html lang="en">
<head>
<link crossorigin="" href="https://static.goat.com" rel="preconnect"/>
<link href="https://static.goat.com" rel="dns-prefetch"/>
<link crossorigin="" href="https://image.goat.com" rel="preconnect"/>
<link href="https://image.goat.com" rel="dns-prefetch"/>
<link href="https://static.goat.com/public/web-assets/fonts/fonts.css" rel="stylesheet" type="text/css">
<link href="https://static.goat.com/public/web-assets/main_cdf7ee0f2ede9a6547f1.css" rel="stylesheet" type="text/css">
<link href="/public/web-assets/icons/favicon.ico" rel="icon" type="image/x-icon">
<meta charset="utf-8"/>
<meta content="UKJdW4ASatmO3fEQRgy9oOKOWmy4Mi5DZqtu8vReGTs" name="google-site-verification"/>
<meta content="70d0748bed9a38a632e92fac88c70291" name="p:domain_verify"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="summary" name="twitter:card">
<meta content="@goatapp" name="twitter:creator"/>
<meta content="GOAT" name="twitter:imag

In [ ]:
goat = {}
long_delays = [i for i in range(10,21)]
delays = [i for i in range(3, 20)]

# for i in range(5):
for index, shoe in enumerate(shoes):
    if shoe in goat.keys():
        continue

    try:
        time.sleep(np.random.choice(delays))
        url = 'https://www.goat.com/search?query=' + str(shoe)
        driver.get(url)
        time.sleep(np.random.choice(long_delays))
        driver.find_element_by_css_selector('#root > div.PageLayout__Wrapper-sc-1s1ahmo-0.kiYGas > div.PageLayout__PageAlignment-sc-1s1ahmo-1.gOnmNv > div.SearchGrid__Content-sc-12gzh1q-6.cOHjrc.goat-width.full-width-mobile > div.filter-wrapper.desktop.undefined > div.filter-content-area > div > div > div:nth-child(1) > div > a').click()
        time.sleep(np.random.choice(delays))
        user_agent = get_random_ua()
        headers = {'user-agent': user_agent}
        soupMain = BeautifulSoup(driver.page_source, 'html.parser', headers=headers)
        resale = soupMain.find_all('span', class_='ProductTitlePaneActions__BuyPrice-l1sjea-4 fZVosI')[0].text
        data = list(soupMain.find_all('div', class_='Fact__Details-lpojmp-1 gfClQW'))        

        goat[shoe] = {'resale_price':resale, 'data':colorway}
        time.sleep(np.random.choice(delays))
    except:
        pass

/Users/nikhilbhargava/opt/anaconda3/envs/mids-python/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
driver.quit()

In [ ]:
with open(raw / 'goat.json', 'w') as outfile:
    json.dump(goat, outfile)